<a href="https://colab.research.google.com/github/thanhbrandon/Song-Generator/blob/main/SongGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Rush Song Generator
by Thanh Tran

Hello, this purpose of this program is generate Rush Songs using previous Rush Songs.

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer

In [10]:
verses = [] # Create Verses String List

# Adds verses from file into list
with open('/content/drive/MyDrive/Rush Album Lyrics/Permanent Waves/Freewill.txt',encoding='cp1252') as f:
    for line in f:
      verses.append(line)


There are those who think that life



In [11]:
tokenizer = Tokenizer(num_words = 100)
tokenizer.fit_on_texts(verses)
word_index = tokenizer.word_index
print(word_index)


{'a': 1, 'of': 2, 'you': 3, 'choose': 4, 'in': 5, 'to': 6, 'the': 7, 'are': 8, 'that': 9, 'can': 10, 'will': 11, 'there': 12, 'those': 13, 'who': 14, 'think': 15, 'with': 16, 'dance': 17, 'we': 18, 'on': 19, 'and': 20, 'i': 21, 'that’s': 22, 'they': 23, 'were': 24, 'life': 25, 'has': 26, 'nothing': 27, 'left': 28, 'chance': 29, 'host': 30, 'holy': 31, 'horrors': 32, 'direct': 33, 'our': 34, 'aimless': 35, 'planet': 36, 'playthings': 37, 'strings': 38, 'powers': 39, 'cannot': 40, 'perceive': 41, '“the': 42, 'stars': 43, 'aren’t': 44, 'aligned': 45, '–': 46, 'or': 47, 'gods': 48, 'malign”': 49, 'blame': 50, 'is': 51, 'better': 52, 'give': 53, 'than': 54, 'receive': 55, 'ready': 56, 'guide': 57, 'some': 58, 'celestial': 59, 'voice': 60, 'if': 61, 'not': 62, 'decide': 63, 'still': 64, 'have': 65, 'made': 66, 'choice': 67, 'from': 68, 'phantom': 69, 'fears': 70, 'kindness': 71, 'kill': 72, 'path': 73, 'clear': 74, 'free': 75, 'dealt': 76, 'losing': 77, 'hand': 78, 'cards': 79, 'stacked': 80